#  T M V A Classification Category Application
This macro provides a simple example on how to use the trained classifiers
(with categories) within an analysis module
- Project   : TMVA - a Root-integrated toolkit for multivariate data analysis
- Package   : TMVA
- Exectuable: TMVAClassificationCategoryApplication



**Author:** Andreas Hoecker  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:46 AM.</small></i>

In [1]:
%%cpp -d
#include <cstdlib>
#include <vector>
#include <iostream>
#include <map>
#include <string>

#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TSystem.h"
#include "TROOT.h"
#include "TH1F.h"
#include "TStopwatch.h"

#if not defined(__CINT__) || defined(__MAKECINT__)
#include "TMVA/Tools.h"
#include "TMVA/Reader.h"
#include "TMVA/MethodCuts.h"
#endif

Two types of category methods are implemented

In [2]:
Bool_t UseOffsetMethod = kTRUE;

---------------------------------------------------------------
 default MVA methods to be trained + tested

In [3]:
std::map<std::string,int> Use;

In [4]:
Use["LikelihoodCat"] = 1;
Use["FisherCat"]     = 1;

---------------------------------------------------------------

In [5]:
std::cout << std::endl
          << "==> Start TMVAClassificationCategoryApplication" << std::endl;


==> Start TMVAClassificationCategoryApplication


Create the reader object

In [6]:
TMVA::Reader *reader = new TMVA::Reader( "!Color:!Silent" );

Create a set of variables and spectators and declare them to the reader
 - the variable names MUST corresponds in name and type to those given in the weight file(s) used

In [7]:
Float_t var1, var2, var3, var4, eta;
reader->AddVariable( "var1", &var1 );
reader->AddVariable( "var2", &var2 );
reader->AddVariable( "var3", &var3 );
reader->AddVariable( "var4", &var4 );

reader->AddSpectator( "eta", &eta );

Book the mva methods

In [8]:
for (std::map<std::string,int>::iterator it = Use.begin(); it != Use.end(); it++) {
   if (it->second) {
      TString methodName = it->first + " method";
      TString weightfile = "dataset/weights/TMVAClassificationCategory_" + TString(it->first) + ".weights.xml";
      reader->BookMVA( methodName, weightfile );
   }
}

                         : Booking "FisherCat method" of type "Category" from dataset/weights/TMVAClassificationCategory_FisherCat.weights.xml.
                         : Reading weight file: dataset/weights/TMVAClassificationCategory_FisherCat.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Recreating sub-classifiers from XML-file 
<HEADER> DataSetInfo              : [Category_Fisher_1_dsi] : Added class "Signal"
<HEADER> DataSetInfo              : [Category_Fisher_1_dsi] : Added class "Background"
<HEADER> DataSetInfo              : [Category_Fisher_2_dsi] : Added class "Signal"
<HEADER> DataSetInfo              : [Category_Fisher_2_dsi] : Added class "Background"
                         : Booked classifier "FisherCat" of type: "Category"
                         : Booking "LikelihoodCat method" of type "Category" from dataset/weights/TMVAClassificatio

Book output histograms

In [9]:
UInt_t nbin = 100;
std::map<std::string,TH1*> hist;
hist["LikelihoodCat"] = new TH1F( "MVA_LikelihoodCat",   "MVA_LikelihoodCat", nbin, -1, 0.9999 );
hist["FisherCat"]     = new TH1F( "MVA_FisherCat",       "MVA_FisherCat",     nbin, -4, 4 );

Prepare input tree (this must be replaced by your data source)
 in this example, there is a toy tree with signal and one with background events
 we'll later on use only the "signal" events for the test in this example.

In [10]:
TString fname = TString(gSystem->DirName(__FILE__) ) + "/data/";

If directory data not found try using tutorials dir

In [11]:
if (gSystem->AccessPathName( fname + "toy_sigbkg_categ_offset.root"  )) {
   fname = gROOT->GetTutorialDir() + "/tmva/data/";
}
if (UseOffsetMethod) fname += "toy_sigbkg_categ_offset.root";
else                 fname += "toy_sigbkg_categ_varoff.root";
std::cout << "--- TMVAClassificationApp    : Accessing " << fname << "!" << std::endl;
TFile *input = TFile::Open(fname);
if (!input) {
   std::cout << "ERROR: could not open data file: " << fname << std::endl;
   exit(1);
}

--- TMVAClassificationApp    : Accessing /mnt/build/workspace/root-makedoc-v618/rootspi/rdoc/src/v6-18-00-patches.build/tutorials/tmva/data/toy_sigbkg_categ_offset.root!


Event loop

Prepare the tree
 - here the variable names have to corresponds to your tree
 - you can use the same variables as above which is slightly faster,
   but of course you can use different ones and copy the values inside the event loop

In [12]:
TTree* theTree = (TTree*)input->Get("TreeS");
std::cout << "--- Use signal sample for evalution" << std::endl;
theTree->SetBranchAddress( "var1", &var1 );
theTree->SetBranchAddress( "var2", &var2 );
theTree->SetBranchAddress( "var3", &var3 );
theTree->SetBranchAddress( "var4", &var4 );

theTree->SetBranchAddress( "eta",  &eta ); // spectator

std::cout << "--- Processing: " << theTree->GetEntries() << " events" << std::endl;
TStopwatch sw;
sw.Start();
for (Long64_t ievt=0; ievt<theTree->GetEntries();ievt++) {

   if (ievt%1000 == 0) std::cout << "--- ... Processing event: " << ievt << std::endl;

   theTree->GetEntry(ievt);

   // Return the MVA outputs and fill into histograms

   for (std::map<std::string,int>::iterator it = Use.begin(); it != Use.end(); it++) {
      if (!it->second) continue;
      TString methodName = it->first + " method";
      hist[it->first]->Fill( reader->EvaluateMVA( methodName ) );
   }

}
sw.Stop();
std::cout << "--- End of event loop: "; sw.Print();

--- Use signal sample for evalution
--- Processing: 10000 events
--- ... Processing event: 0
--- ... Processing event: 1000
--- ... Processing event: 2000
--- ... Processing event: 3000
--- ... Processing event: 4000
--- ... Processing event: 5000
--- ... Processing event: 6000
--- ... Processing event: 7000
--- ... Processing event: 8000
--- ... Processing event: 9000
--- End of event loop: Real time 0:00:00, CP time 0.030


Write histograms

In [13]:
TFile *target  = new TFile( "TMVApp.root","RECREATE" );
for (std::map<std::string,int>::iterator it = Use.begin(); it != Use.end(); it++)
   if (it->second) hist[it->first]->Write();

target->Close();
std::cout << "--- Created root file: \"TMVApp.root\" containing the MVA output histograms" << std::endl;

delete reader;
std::cout << "==> TMVAClassificationApplication is done!" << std::endl << std::endl;

--- Created root file: "TMVApp.root" containing the MVA output histograms
==> TMVAClassificationApplication is done!

